## Шаг 2. Предобработка данных

In [1]:
import pandas as pd
import numpy as np
from functools import reduce

import warnings

warnings.filterwarnings("ignore")

### Импортируем получившиеся датафреймы

In [2]:
IMOEX_df_before_EDA = pd.read_csv('IMOEX_df_before_EDA')
MOEXMM_df_before_EDA = pd.read_csv('MOEXMM_df_before_EDA')
GOLD_df_before_EDA = pd.read_csv('GOLD_df_before_EDA')
USDRUB_df_before_EDA = pd.read_csv('USDRUB_df_before_EDA')
INFLATION_df_before_EDA = pd.read_csv('INFLATION_df_before_EDA')

### Обработка датафрейма IMOEX

In [3]:
IMOEX_df_before_EDA.head()

,BOARDID,SECID,TRADEDATE,SHORTNAME,NAME,CLOSE,OPEN,HIGH,LOW,VALUE,DURATION,YIELD,DECIMALS,CAPITALIZATION,CURRENCYID,DIVISOR,TRADINGSESSION,VOLUME
0,SNDX,IMOEX,2013-09-02,Индекс МосБиржи,Индекс МосБиржи,1367.53,1364.75,1373.73,1364.27,1.274407e+10,NaN,NaN,2,6.096417e+12,RUB,NaN,3,NaN
1,SNDX,IMOEX,2013-09-03,Индекс МосБиржи,Индекс МосБиржи,1373.82,1366.99,1384.18,1366.99,2.216425e+10,NaN,NaN,2,6.124461e+12,RUB,NaN,3,NaN
2,SNDX,IMOEX,2013-09-04,Индекс МосБиржи,Индекс МосБиржи,1375.66,1372.98,1376.42,1365.21,1.692969e+10,NaN,NaN,2,6.132675e+12,RUB,NaN,3,NaN
3,SNDX,IMOEX,2013-09-05,Индекс МосБиржи,Индекс МосБиржи,1422.40,1375.66,1422.40,1373.29,3.763559e+10,NaN,NaN,2,6.341025e+12,RUB,NaN,3,NaN
4,SNDX,IMOEX,2013-09-06,Индекс МосБиржи,Индекс МосБиржи,1423.40,1421.86,1448.75,1418.80,5.218912e+10,NaN,NaN,2,6.345500e+12,RUB,NaN,3,NaN


Посмотрим количество пропусков в каждом столбце

In [4]:
IMOEX_df_before_EDA.isnull().sum()

BOARDID              0
SECID                0
TRADEDATE            0
SHORTNAME            0
NAME                 0
CLOSE                0
OPEN                 0
HIGH                 0
LOW                  0
VALUE                0
DURATION          1065
YIELD             1065
DECIMALS             0
CAPITALIZATION       0
CURRENCYID           0
DIVISOR             80
TRADINGSESSION       0
VOLUME            2845
dtype: int64

In [5]:
print(IMOEX_df_before_EDA['DURATION'].unique())
print(IMOEX_df_before_EDA['YIELD'].unique())
print(IMOEX_df_before_EDA['DIVISOR'].unique())

[nan  0.]
[nan  0.]
[           nan 4.51876015e+09 4.51289187e+09 4.50866224e+09
 4.76294583e+09 4.63550338e+09 4.69729697e+09 4.68588832e+09
 4.66370056e+09 4.63258322e+09 4.68972910e+09 4.69700973e+09
 4.60042769e+09 4.72701074e+09 4.68707420e+09 4.78260596e+09
 4.63467912e+09 4.59665934e+09 4.59665934e+09 4.40892573e+09
 4.31169756e+09 4.31523349e+09 4.27786628e+09 4.26282623e+09
 4.14158796e+09 3.94901864e+09 3.70878002e+09 3.92779150e+09
 3.87151407e+09 4.04285118e+09 4.25783933e+09 4.80970779e+09
 4.80970779e+09 5.07923087e+09 5.07733607e+09 5.10549752e+09
 5.02319112e+09 4.94484716e+09 4.78149429e+09 4.77784852e+09
 4.93014674e+09 4.87287880e+09 3.41631366e+09 1.69197187e+09
 1.74596130e+09 1.77104960e+09 1.76843274e+09 1.76431300e+09
 1.84096879e+09 1.98085495e+09 1.97520662e+09 1.95111325e+09
 1.93733002e+09 2.16270349e+09]


* Признаки в DURATION и YIELD в 1065 случаях заполнены "NaN", в оставшихся - заполнены нулями.

* Признак DIVISOR - в 80 случаях заполнен "NaN". Однако его значение учитывается в расчете самого индекса, поэтому значения делителя индекса для анализа не имеют значения.

#### Описания признаков до исключения

Для каждого из датафреймов индексов московской биржи названия столбцов совпадают, поэтому здесь я опишу каждый из столбцов до исключения каких-либо из них

* **BOARDID** - идентификатор торговой площадки и режима торгов, в данном случае торговая площадка для всех данных индекса - Московская биржа, режим торгов - SNDX - то есть индекс. Этот показатель не нужен для анализа, потому что совпадает на всех данных.

* **SECID** - уникальный идентификатор инструмента, присваиваемый биржей. В данном случае это название индекса - IMOEX, аналогично у других индексов. Нам он также не нужен для анализа, потому что совпадает для конкретного датасета у всех данных.

* **TRADEDATE** - дата, на которую были получены данные. Этот столбец необходим, чтобы в дальнейшем объединить данные в один дата фрейм.

* **SHORTNAME** - краткое название инструмента, в случае индексов, как правило совпадает с NAME, поэтому этот столбец не нужен.

* **NAME** - полное название инструмента. Однако название инструмента выгодно поместить в заголовки тех столбцов, которые не будут удалены, чтобы обращаться к этим столбцам заранее понимая к какому индексу они относятся, поэтому мы удалим этот столбец, а имя инструмента внесем в название нужных нам столбцов.

* **CLOSE** - цена закрытия - применимый к анализу столбец

* **OPEN** - цена открытия - применимый к анализу столбец

* **HIGH** - максимальная цена за время торгов - применимый к анализу столбец

* **LOW**  - минимальная цена за время торгов - применимый к анализу столбец

* **VALUE** - объем торгов - применимый к анализу столбец

* **DURATION** - в данном случае не нужен для анализа, так как термин дюрация применяется как правило к облигациям, и соответственно к облигационным индексам (RGBI), в индексах акций она не нужна.

* **YIELD** - аналогично с дюрацией - не применимый к индексу показатель (под yield имеется в виду купонная или дивидендная доходность, однако в данном случае к индексу они не применимы)

* **DECIMALS** - количество знаков после запятой при отображении цены инструмента. Этот столбец будет исключен

* **CAPITALIZATION** - оценивается по расчётному весу бумаг в итоговом значении индекса, применимый к анализу столбец

* **CURRENCYID** - валюта, в которой отображается индекс. Все индексы из всех датафреймов - российские и измеряются в рублях, поэтому этот столбец будет удален

* **DIVISOR** - это число, выбранное при создании индекса фондового рынка, взвешенного по цене, которое применяется к индексу для создания более удобного для восприятия значения индекса. 

* **TRADINGSESSION**  - акции, входящие в индекс Мосбиржи торгуются в вечернуюю сессию (то есть = 3), поэтому во всем фрейме значение столбца будет = 3. Этот столбец исключим

* **VOLUME** - значение, полностью заполненное "NaN", поэтому столбец будет исключен

In [6]:
Subset = ['TRADEDATE', 'CLOSE', 'OPEN', 'HIGH', 'LOW', 'VALUE', 'CAPITALIZATION']
IMOEX_df = IMOEX_df_before_EDA[Subset]
Headings_IMOEX = {
    'TRADEDATE': 'TRADEDATE',
    'CLOSE': 'CLOSE_IMOEX',
    'OPEN': 'OPEN_IMOEX',
    'HIGH': 'HIGH_IMOEX',
    'LOW': 'LOW_IMOEX',
    'VALUE': 'VALUE_IMOEX',
    'CAPITALIZATION': 'CAPITALIZATION_IMOEX'
}
IMOEX_df = IMOEX_df.rename(columns=Headings_IMOEX)
print(IMOEX_df.shape)
IMOEX_df.head()

(2845, 7)


,TRADEDATE,CLOSE_IMOEX,OPEN_IMOEX,HIGH_IMOEX,LOW_IMOEX,VALUE_IMOEX,CAPITALIZATION_IMOEX
0,2013-09-02,1367.53,1364.75,1373.73,1364.27,1.274407e+10,6.096417e+12
1,2013-09-03,1373.82,1366.99,1384.18,1366.99,2.216425e+10,6.124461e+12
2,2013-09-04,1375.66,1372.98,1376.42,1365.21,1.692969e+10,6.132675e+12
3,2013-09-05,1422.40,1375.66,1422.40,1373.29,3.763559e+10,6.341025e+12
4,2013-09-06,1423.40,1421.86,1448.75,1418.80,5.218912e+10,6.345500e+12


В получившемся итоговом датафрейме пропусков не осталось

In [7]:
IMOEX_df.isnull().any()

TRADEDATE               False
CLOSE_IMOEX             False
OPEN_IMOEX              False
HIGH_IMOEX              False
LOW_IMOEX               False
VALUE_IMOEX             False
CAPITALIZATION_IMOEX    False
dtype: bool

### Обработка MOEX_MM

In [8]:
MOEXMM_df_before_EDA.head()

,BOARDID,SECID,TRADEDATE,SHORTNAME,NAME,CLOSE,OPEN,HIGH,LOW,VALUE,DURATION,YIELD,DECIMALS,CAPITALIZATION,CURRENCYID,DIVISOR,TRADINGSESSION,VOLUME
0,SNDX,MOEXMM,2013-09-02,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2109.53,2103.86,2120.72,2100.19,8.886807e+08,NaN,NaN,2,2.345484e+11,RUB,NaN,3,NaN
1,SNDX,MOEXMM,2013-09-03,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2102.51,2109.79,2123.56,2089.09,1.304529e+09,NaN,NaN,2,2.337673e+11,RUB,NaN,3,NaN
2,SNDX,MOEXMM,2013-09-04,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2109.74,2101.98,2112.03,2088.95,1.283077e+09,NaN,NaN,2,2.345715e+11,RUB,NaN,3,NaN
3,SNDX,MOEXMM,2013-09-05,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2161.35,2110.70,2162.06,2104.88,2.779851e+09,NaN,NaN,2,2.403100e+11,RUB,NaN,3,NaN
4,SNDX,MOEXMM,2013-09-06,Индекс металлов и добычи,Индекс МосБиржи металлов и добычи,2157.45,2159.63,2199.92,2154.78,3.672785e+09,NaN,NaN,2,2.398759e+11,RUB,NaN,3,NaN


In [9]:
MOEXMM_df_before_EDA.isnull().sum()

BOARDID              0
SECID                0
TRADEDATE            0
SHORTNAME            0
NAME                 0
CLOSE                0
OPEN                 0
HIGH                 0
LOW                  0
VALUE                0
DURATION          1065
YIELD             1065
DECIMALS             0
CAPITALIZATION       0
CURRENCYID           0
DIVISOR             80
TRADINGSESSION       0
VOLUME            2842
dtype: int64

In [10]:
print(MOEXMM_df_before_EDA['DURATION'].unique())
print(MOEXMM_df_before_EDA['YIELD'].unique())
print(MOEXMM_df_before_EDA['DIVISOR'].unique())

[nan  0.]
[nan  0.]
[           nan 1.62353092e+08 1.44537357e+08 1.26491770e+08
 1.27133103e+08 1.27432336e+08 1.39922642e+08 1.34654768e+08
 1.35272929e+08 1.20986892e+08 1.12702897e+08 1.29080340e+08
 1.30446110e+08 1.34221324e+08 1.21584398e+08 1.57140921e+08
 1.59587492e+08 1.79722214e+08 1.79722214e+08 1.79575136e+08
 1.77827615e+08 1.69385852e+08 1.61823138e+08 1.69285623e+08
 1.72755994e+08 1.69563875e+08 1.79400138e+08 1.84464611e+08
 1.72138181e+08 1.58483725e+08 1.60529704e+08 1.60529704e+08
 1.77542508e+08 1.98708016e+08 2.06122011e+08 2.22341609e+08
 2.25196183e+08 2.08936150e+08 2.07648058e+08 2.12555730e+08
 2.14727821e+08 1.63382893e+08 1.00527157e+08 1.12468175e+08
 1.07058846e+08 1.08257207e+08 1.06246719e+08 1.00865142e+08
 9.96375796e+07]


In [11]:
Subset = ['TRADEDATE', 'CLOSE', 'OPEN', 'HIGH', 'LOW', 'VALUE', 'CAPITALIZATION']
MOEXMM_df = MOEXMM_df_before_EDA[Subset]
Headings_MOEXMM = {
    'TRADEDATE': 'TRADEDATE',
    'CLOSE': 'CLOSE_MOEXMM',
    'OPEN': 'OPEN_MOEXMM',
    'HIGH': 'HIGH_MOEXMM',
    'LOW': 'LOW_MOEXMM',
    'VALUE': 'VALUE_MOEXMM',
    'CAPITALIZATION': 'CAPITALIZATION_MOEXMM'
}
MOEXMM_df = MOEXMM_df.rename(columns=Headings_MOEXMM)
print(MOEXMM_df.shape)
MOEXMM_df.head()

(2842, 7)


,TRADEDATE,CLOSE_MOEXMM,OPEN_MOEXMM,HIGH_MOEXMM,LOW_MOEXMM,VALUE_MOEXMM,CAPITALIZATION_MOEXMM
0,2013-09-02,2109.53,2103.86,2120.72,2100.19,8.886807e+08,2.345484e+11
1,2013-09-03,2102.51,2109.79,2123.56,2089.09,1.304529e+09,2.337673e+11
2,2013-09-04,2109.74,2101.98,2112.03,2088.95,1.283077e+09,2.345715e+11
3,2013-09-05,2161.35,2110.70,2162.06,2104.88,2.779851e+09,2.403100e+11
4,2013-09-06,2157.45,2159.63,2199.92,2154.78,3.672785e+09,2.398759e+11


In [12]:
MOEXMM_df.isnull().any()

TRADEDATE                False
CLOSE_MOEXMM             False
OPEN_MOEXMM              False
HIGH_MOEXMM              False
LOW_MOEXMM               False
VALUE_MOEXMM             False
CAPITALIZATION_MOEXMM    False
dtype: bool

### Обработка датафрейма с данными по инфляции

In [13]:
INFLATION_df_before_EDA.head()

,Date,Key_rate,Inflation,Inflation_goal
0,2024-12-01,"21,00","9,52","4,00"
1,2024-11-01,"21,00","8,88","4,00"
2,2024-10-01,"21,00","8,54","4,00"
3,2024-09-01,"19,00","8,63","4,00"
4,2024-08-01,"18,00","9,05","4,00"


### Описание признаков до исключения

* **Key_rate** - ключевая ставка, устанавливаемая ЦБ. 

* **Inflation** - среднемесячный темп инфляции. Этот признак особо важен, так как золото является хеджирующим активом от инфляции, следовательно, именно этот показатель будет формировать основную динамику цен на золото

* **Inflational_goal** - таргет по инфляции 

В столбцах Key_rate, Inflation, Inflation_goal значения нельзя конвертировать во float, так как там стоит запятая как разделитель. Реализуем функцию которая меняет формат этих данных

In [14]:
def correct(number):
    cor_val = number.replace(',', '.')
    return cor_val


corrected_1 = INFLATION_df_before_EDA['Key_rate'].map(correct)
INFLATION_df_before_EDA['Key_rate'] = corrected_1
corrected_2 = INFLATION_df_before_EDA['Inflation'].map(correct)
INFLATION_df_before_EDA['Inflation'] = corrected_2
corrected_3 = INFLATION_df_before_EDA['Inflation_goal'].map(correct)
INFLATION_df_before_EDA['Inflation_goal'] = corrected_3
INFLATION_df_before_EDA.head()


,Date,Key_rate,Inflation,Inflation_goal
0,2024-12-01,21.00,9.52,4.00
1,2024-11-01,21.00,8.88,4.00
2,2024-10-01,21.00,8.54,4.00
3,2024-09-01,19.00,8.63,4.00
4,2024-08-01,18.00,9.05,4.00


In [15]:
INFLATION_df_before_EDA['Inflation_goal'].unique()

array(['4.00', '—'], dtype=object)

Таргет по инфляции одинаковый для всего дата фрейма, ниже 2014 года он заполнен NaN, так как тогда еще не проводилась политика таргетирования инфляции, поэтому исключим этот столбец

In [16]:
INFLATION_df_before_EDA = INFLATION_df_before_EDA.drop(['Inflation_goal'], axis=1)
INFLATION_df_before_EDA.rename(columns={'Date': 'TRADEDATE'}, inplace=True)
INFLATION_df_before_EDA.isnull().any()

TRADEDATE    False
Key_rate     False
Inflation    False
dtype: bool

Не все данные отсортированы по дате: GOLD_df USDRUB_df, MOEXMM, IMOEX отсортированы от start_date, датафрейм инфляции, наоборот, начинается с end_date. Поэтому нам надо отсортировать INFLATION по дате

In [17]:
INFLATION_df = INFLATION_df_before_EDA.sort_values(by='TRADEDATE')
print(INFLATION_df.shape)
INFLATION_df

(136, 3)


,TRADEDATE,Key_rate,Inflation
135,2013-09-01,5.50,6.14
134,2013-10-01,5.50,6.27
133,2013-11-01,5.50,6.50
132,2013-12-01,5.50,6.47
131,2014-01-01,5.50,6.07
...,...,...,...
4,2024-08-01,18.00,9.05
3,2024-09-01,19.00,8.63
2,2024-10-01,21.00,8.54
1,2024-11-01,21.00,8.88


### Обработка датафрейма валютного курса USD RUB из данных ЦБ

Признаком является официальный курс Банка России валютной пары  доллар рубль 

In [18]:
USDRUB_df_before_EDA.head()

,Date,USD_RUB
0,03.09.2013,33.2522
1,04.09.2013,33.3693
2,05.09.2013,33.4656
3,06.09.2013,33.3901
4,07.09.2013,33.4338


Единственной проблемой в этом датафрейме является тот факт, что дата не находится в формате date.


In [19]:
USDRUB_df_before_EDA['Date'] = pd.to_datetime(USDRUB_df_before_EDA['Date'], format='%d.%m.%Y')
USDRUB_df_before_EDA.rename(columns={'Date': 'TRADEDATE'}, inplace=True)
print(USDRUB_df_before_EDA.shape)
USDRUB_df_before_EDA.head()

(2800, 2)


,TRADEDATE,USD_RUB
0,2013-09-03,33.2522
1,2013-09-04,33.3693
2,2013-09-05,33.4656
3,2013-09-06,33.3901
4,2013-09-07,33.4338


In [20]:
print(USDRUB_df_before_EDA.isnull().any())
USDRUB_df = USDRUB_df_before_EDA
print(USDRUB_df.shape)
USDRUB_df.head()

TRADEDATE    False
USD_RUB      False
dtype: bool
(2800, 2)


,TRADEDATE,USD_RUB
0,2013-09-03,33.2522
1,2013-09-04,33.3693
2,2013-09-05,33.4656
3,2013-09-06,33.3901
4,2013-09-07,33.4338


### Обработка датафрейма цен на золото из данных ЦБ

In [21]:
GOLD_df_before_EDA.head()

,Date,Price_GOLD
0,03.09.2013,"1487,36"
1,04.09.2013,"1492,6"
2,05.09.2013,"1510,36"
3,06.09.2013,"1494,07"
4,07.09.2013,"1470,76"


In [22]:
GOLD_df_before_EDA.isnull().any()

Date          False
Price_GOLD    False
dtype: bool

In [23]:
GOLD_df_before_EDA['Date'] = pd.to_datetime(GOLD_df_before_EDA['Date'], format='%d.%m.%Y')
GOLD_df_before_EDA.rename(columns={'Date': 'TRADEDATE'},  inplace=True)
GOLD_df_before_EDA.head()

,TRADEDATE,Price_GOLD
0,2013-09-03,"1487,36"
1,2013-09-04,"1492,6"
2,2013-09-05,"1510,36"
3,2013-09-06,"1494,07"
4,2013-09-07,"1470,76"


In [24]:
def correct(number):
    cor_val = number.replace(',', '.')
    return cor_val


corrected = GOLD_df_before_EDA['Price_GOLD'].map(correct)
GOLD_df_before_EDA['Price_GOLD'] = corrected
GOLD_df = GOLD_df_before_EDA
print(GOLD_df.shape)
GOLD_df.head()

(2800, 2)


,TRADEDATE,Price_GOLD
0,2013-09-03,1487.36
1,2013-09-04,1492.6
2,2013-09-05,1510.36
3,2013-09-06,1494.07
4,2013-09-07,1470.76


### Объединение датафреймов

In [25]:
print(IMOEX_df['TRADEDATE'].duplicated().sum())  # Проверка дубликатов в IMOEX
print(MOEXMM_df['TRADEDATE'].duplicated().sum())  # Проверка дубликатов в MOEXMM

0
0


In [26]:
# во избежание ошибок еще раз перевод столбца TRADEDATE в формат дат
IMOEX_df['TRADEDATE'] = pd.to_datetime(IMOEX_df['TRADEDATE'])
MOEXMM_df['TRADEDATE'] = pd.to_datetime(MOEXMM_df['TRADEDATE'])
INFLATION_df['TRADEDATE'] = pd.to_datetime(INFLATION_df['TRADEDATE'])
USDRUB_df['TRADEDATE'] = pd.to_datetime(USDRUB_df['TRADEDATE'])
GOLD_df['TRADEDATE'] = pd.to_datetime(GOLD_df['TRADEDATE'])

dataframes = [IMOEX_df, MOEXMM_df, INFLATION_df, USDRUB_df, GOLD_df]

# функция для объединения (функция reduce из модуля functools последовательно кумулятивно берет элементы из dataframes и объединяет их слева направо)
RESULT_df = reduce(lambda x, y: pd.merge(x, y, on='TRADEDATE', how='outer'), dataframes)

print(RESULT_df.shape)
RESULT_df.head(30)

(3483, 17)


,TRADEDATE,CLOSE_IMOEX,OPEN_IMOEX,HIGH_IMOEX,LOW_IMOEX,VALUE_IMOEX,CAPITALIZATION_IMOEX,CLOSE_MOEXMM,OPEN_MOEXMM,HIGH_MOEXMM,LOW_MOEXMM,VALUE_MOEXMM,CAPITALIZATION_MOEXMM,Key_rate,Inflation,USD_RUB,Price_GOLD
0,2013-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.50,6.14,NaN,NaN
1,2013-09-02,1367.53,1364.75,1373.73,1364.27,1.274407e+10,6.096417e+12,2109.53,2103.86,2120.72,2100.19,8.886807e+08,2.345484e+11,NaN,NaN,NaN,NaN
2,2013-09-03,1373.82,1366.99,1384.18,1366.99,2.216425e+10,6.124461e+12,2102.51,2109.79,2123.56,2089.09,1.304529e+09,2.337673e+11,NaN,NaN,33.2522,1487.36
3,2013-09-04,1375.66,1372.98,1376.42,1365.21,1.692969e+10,6.132675e+12,2109.74,2101.98,2112.03,2088.95,1.283077e+09,2.345715e+11,NaN,NaN,33.3693,1492.6
4,2013-09-05,1422.40,1375.66,1422.40,1373.29,3.763559e+10,6.341025e+12,2161.35,2110.70,2162.06,2104.88,2.779851e+09,2.403100e+11,NaN,NaN,33.4656,1510.36
5,2013-09-06,1423.40,1421.86,1448.75,1418.80,5.218912e+10,6.345500e+12,2157.45,2159.63,2199.92,2154.78,3.672785e+09,2.398759e+11,NaN,NaN,33.3901,1494.07
6,2013-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.4338,1470.76
7,2013-09-09,1451.54,1423.40,1455.28,1421.45,4.579322e+10,6.470954e+12,2191.94,2164.13,2199.41,2164.13,3.046784e+09,2.437106e+11,NaN,NaN,NaN,NaN
8,2013-09-10,1453.57,1451.54,1464.20,1446.97,4.892904e+10,6.480001e+12,2234.30,2195.60,2238.43,2195.60,3.415190e+09,2.484205e+11,NaN,NaN,33.3243,1484.96
9,2013-09-11,1454.12,1452.52,1459.68,1449.58,3.068467e+10,6.482454e+12,2205.88,2234.30,2238.07,2204.98,2.719825e+09,2.452612e+11,NaN,NaN,33.0600,1459.37


Добавим качественный признак - выходной или рабочий день:

In [27]:
RESULT_df['Day_type'] = np.where(RESULT_df['CLOSE_IMOEX'].isna(), 'выходной', 'рабочий день')
RESULT_df.head(10)

,TRADEDATE,CLOSE_IMOEX,OPEN_IMOEX,HIGH_IMOEX,LOW_IMOEX,VALUE_IMOEX,CAPITALIZATION_IMOEX,CLOSE_MOEXMM,OPEN_MOEXMM,HIGH_MOEXMM,LOW_MOEXMM,VALUE_MOEXMM,CAPITALIZATION_MOEXMM,Key_rate,Inflation,USD_RUB,Price_GOLD,Day_type
0,2013-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.50,6.14,NaN,NaN,выходной
1,2013-09-02,1367.53,1364.75,1373.73,1364.27,1.274407e+10,6.096417e+12,2109.53,2103.86,2120.72,2100.19,8.886807e+08,2.345484e+11,NaN,NaN,NaN,NaN,рабочий день
2,2013-09-03,1373.82,1366.99,1384.18,1366.99,2.216425e+10,6.124461e+12,2102.51,2109.79,2123.56,2089.09,1.304529e+09,2.337673e+11,NaN,NaN,33.2522,1487.36,рабочий день
3,2013-09-04,1375.66,1372.98,1376.42,1365.21,1.692969e+10,6.132675e+12,2109.74,2101.98,2112.03,2088.95,1.283077e+09,2.345715e+11,NaN,NaN,33.3693,1492.6,рабочий день
4,2013-09-05,1422.40,1375.66,1422.40,1373.29,3.763559e+10,6.341025e+12,2161.35,2110.70,2162.06,2104.88,2.779851e+09,2.403100e+11,NaN,NaN,33.4656,1510.36,рабочий день
5,2013-09-06,1423.40,1421.86,1448.75,1418.80,5.218912e+10,6.345500e+12,2157.45,2159.63,2199.92,2154.78,3.672785e+09,2.398759e+11,NaN,NaN,33.3901,1494.07,рабочий день
6,2013-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.4338,1470.76,выходной
7,2013-09-09,1451.54,1423.40,1455.28,1421.45,4.579322e+10,6.470954e+12,2191.94,2164.13,2199.41,2164.13,3.046784e+09,2.437106e+11,NaN,NaN,NaN,NaN,рабочий день
8,2013-09-10,1453.57,1451.54,1464.20,1446.97,4.892904e+10,6.480001e+12,2234.30,2195.60,2238.43,2195.60,3.415190e+09,2.484205e+11,NaN,NaN,33.3243,1484.96,рабочий день
9,2013-09-11,1454.12,1452.52,1459.68,1449.58,3.068467e+10,6.482454e+12,2205.88,2234.30,2238.07,2204.98,2.719825e+09,2.452612e+11,NaN,NaN,33.0600,1459.37,рабочий день


### Заполнение пропусков

In [28]:
RESULT_df.isnull().sum()

TRADEDATE                   0
CLOSE_IMOEX               638
OPEN_IMOEX                638
HIGH_IMOEX                638
LOW_IMOEX                 638
VALUE_IMOEX               638
CAPITALIZATION_IMOEX      638
CLOSE_MOEXMM              641
OPEN_MOEXMM               641
HIGH_MOEXMM               641
LOW_MOEXMM                641
VALUE_MOEXMM              641
CAPITALIZATION_MOEXMM     641
Key_rate                 3347
Inflation                3347
USD_RUB                   683
Price_GOLD                683
Day_type                    0
dtype: int64

Если в день нет данных по индексам, это означает что день - выходной. В данном случае разумно было бы предположить, что цена за выходной будет средним между ценами в смежные рабочие дни, поэтому эти пропуски мы заполним средним значением. Также составим новую колонку, где будет прописано рабочий это или выходной день.

In [29]:
print(RESULT_df.dtypes)

TRADEDATE                datetime64[ns]
CLOSE_IMOEX                     float64
OPEN_IMOEX                      float64
HIGH_IMOEX                      float64
LOW_IMOEX                       float64
VALUE_IMOEX                     float64
CAPITALIZATION_IMOEX            float64
CLOSE_MOEXMM                    float64
OPEN_MOEXMM                     float64
HIGH_MOEXMM                     float64
LOW_MOEXMM                      float64
VALUE_MOEXMM                    float64
CAPITALIZATION_MOEXMM           float64
Key_rate                         object
Inflation                        object
USD_RUB                         float64
Price_GOLD                       object
Day_type                         object
dtype: object


In [30]:
RESULT_df['Price_GOLD'] = pd.to_numeric(RESULT_df['Price_GOLD'])
RESULT_df['Key_rate'] = pd.to_numeric(RESULT_df['Key_rate'])
RESULT_df['Inflation'] = pd.to_numeric(RESULT_df['Inflation'])
print(RESULT_df.dtypes)

TRADEDATE                datetime64[ns]
CLOSE_IMOEX                     float64
OPEN_IMOEX                      float64
HIGH_IMOEX                      float64
LOW_IMOEX                       float64
VALUE_IMOEX                     float64
CAPITALIZATION_IMOEX            float64
CLOSE_MOEXMM                    float64
OPEN_MOEXMM                     float64
HIGH_MOEXMM                     float64
LOW_MOEXMM                      float64
VALUE_MOEXMM                    float64
CAPITALIZATION_MOEXMM           float64
Key_rate                        float64
Inflation                       float64
USD_RUB                         float64
Price_GOLD                      float64
Day_type                         object
dtype: object


In [31]:
numeric_columns = ['CLOSE_IMOEX', 'OPEN_IMOEX', 'HIGH_IMOEX', 'LOW_IMOEX', 'VALUE_IMOEX',
       'CAPITALIZATION_IMOEX', 'CLOSE_MOEXMM', 'OPEN_MOEXMM', 'HIGH_MOEXMM',
       'LOW_MOEXMM', 'VALUE_MOEXMM', 'CAPITALIZATION_MOEXMM', 'USD_RUB', 'Price_GOLD']
       
RESULT_df[numeric_columns] = RESULT_df[numeric_columns].interpolate(method='linear')
RESULT_df = RESULT_df[1:]
RESULT_df['USD_RUB'].iloc[0] = RESULT_df['USD_RUB'].iloc[1]
RESULT_df['Price_GOLD'].iloc[0] = RESULT_df['Price_GOLD'].iloc[1]
RESULT_df.head()

,TRADEDATE,CLOSE_IMOEX,OPEN_IMOEX,HIGH_IMOEX,LOW_IMOEX,VALUE_IMOEX,CAPITALIZATION_IMOEX,CLOSE_MOEXMM,OPEN_MOEXMM,HIGH_MOEXMM,LOW_MOEXMM,VALUE_MOEXMM,CAPITALIZATION_MOEXMM,Key_rate,Inflation,USD_RUB,Price_GOLD,Day_type
1,2013-09-02,1367.53,1364.75,1373.73,1364.27,1.274407e+10,6.096417e+12,2109.53,2103.86,2120.72,2100.19,8.886807e+08,2.345484e+11,NaN,NaN,33.2522,1487.36,рабочий день
2,2013-09-03,1373.82,1366.99,1384.18,1366.99,2.216425e+10,6.124461e+12,2102.51,2109.79,2123.56,2089.09,1.304529e+09,2.337673e+11,NaN,NaN,33.2522,1487.36,рабочий день
3,2013-09-04,1375.66,1372.98,1376.42,1365.21,1.692969e+10,6.132675e+12,2109.74,2101.98,2112.03,2088.95,1.283077e+09,2.345715e+11,NaN,NaN,33.3693,1492.60,рабочий день
4,2013-09-05,1422.40,1375.66,1422.40,1373.29,3.763559e+10,6.341025e+12,2161.35,2110.70,2162.06,2104.88,2.779851e+09,2.403100e+11,NaN,NaN,33.4656,1510.36,рабочий день
5,2013-09-06,1423.40,1421.86,1448.75,1418.80,5.218912e+10,6.345500e+12,2157.45,2159.63,2199.92,2154.78,3.672785e+09,2.398759e+11,NaN,NaN,33.3901,1494.07,рабочий день


Заполним столбцы инфляции и ключевой ставки: инфляцию заполним методом линейной интерполяции, предполагая что инфляция, несмотря на то что измеряется раз в месяц, в течение этого месяца меняется

In [32]:
first_valid_index = RESULT_df['Inflation'].first_valid_index()

RESULT_df['filled_Inflation'] = RESULT_df['Inflation'].copy()
RESULT_df.loc[:first_valid_index, 'filled_Inflation'] = RESULT_df['filled_Inflation'].loc[:first_valid_index].fillna(RESULT_df['Inflation'][first_valid_index])

RESULT_df['filled_Inflation'] = RESULT_df['filled_Inflation'].interpolate()

RESULT_df['Inflation'] = RESULT_df['filled_Inflation']
RESULT_df = RESULT_df.drop(['filled_Inflation'], axis=1)

Значения ключевой ставки заполним ставкой в начале месяца: в отличие от инфляции она фиксирована, поэтому изменения происходят только в начале месяца

In [33]:
def get_month(date):
    return date.month


RESULT_df['month'] = RESULT_df['TRADEDATE'].apply(get_month)
RESULT_df['filled_Key_rate'] = RESULT_df.groupby('month')['Key_rate'].ffill()
RESULT_df['filled_Key_rate'] = RESULT_df.groupby('month')['filled_Key_rate'].bfill()
RESULT_df['Key_rate'] = RESULT_df['filled_Key_rate']
RESULT_df = RESULT_df.drop(['filled_Key_rate'], axis=1)
RESULT_df.head(100)

,TRADEDATE,CLOSE_IMOEX,OPEN_IMOEX,HIGH_IMOEX,LOW_IMOEX,VALUE_IMOEX,CAPITALIZATION_IMOEX,CLOSE_MOEXMM,OPEN_MOEXMM,HIGH_MOEXMM,LOW_MOEXMM,VALUE_MOEXMM,CAPITALIZATION_MOEXMM,Key_rate,Inflation,USD_RUB,Price_GOLD,Day_type,month
1,2013-09-02,1367.53,1364.75,1373.730,1364.27,1.274407e+10,6.096417e+12,2109.53,2103.86,2120.720,2100.19,8.886807e+08,2.345484e+11,8.0,6.270000,33.2522,1487.360,рабочий день,9
2,2013-09-03,1373.82,1366.99,1384.180,1366.99,2.216425e+10,6.124461e+12,2102.51,2109.79,2123.560,2089.09,1.304529e+09,2.337673e+11,8.0,6.270000,33.2522,1487.360,рабочий день,9
3,2013-09-04,1375.66,1372.98,1376.420,1365.21,1.692969e+10,6.132675e+12,2109.74,2101.98,2112.030,2088.95,1.283077e+09,2.345715e+11,8.0,6.270000,33.3693,1492.600,рабочий день,9
4,2013-09-05,1422.40,1375.66,1422.400,1373.29,3.763559e+10,6.341025e+12,2161.35,2110.70,2162.060,2104.88,2.779851e+09,2.403100e+11,8.0,6.270000,33.4656,1510.360,рабочий день,9
5,2013-09-06,1423.40,1421.86,1448.750,1418.80,5.218912e+10,6.345500e+12,2157.45,2159.63,2199.920,2154.78,3.672785e+09,2.398759e+11,8.0,6.270000,33.3901,1494.070,рабочий день,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2013-12-21,1502.36,1498.24,1506.475,1493.10,3.262508e+10,6.788806e+12,2196.23,2207.80,2219.285,2186.08,2.476403e+09,3.565650e+11,5.5,6.203333,32.9798,1267.090,выходной,12
97,2013-12-23,1507.49,1497.31,1510.090,1495.22,2.868477e+10,6.811969e+12,2199.96,2194.13,2210.160,2187.30,1.973058e+09,3.571708e+11,5.5,6.188519,32.9652,1265.335,рабочий день,12
98,2013-12-24,1506.41,1506.89,1510.900,1500.66,1.752864e+10,6.807119e+12,2219.02,2199.24,2226.380,2188.80,1.772672e+09,3.602653e+11,5.5,6.173704,32.9506,1263.580,рабочий день,12
99,2013-12-25,1501.12,1506.41,1510.280,1500.49,1.200722e+10,6.783196e+12,2218.25,2220.00,2224.090,2210.04,1.481828e+09,3.601404e+11,5.5,6.158889,32.6284,1255.160,рабочий день,12


In [34]:
RESULT_df.isnull().any()

TRADEDATE                False
CLOSE_IMOEX              False
OPEN_IMOEX               False
HIGH_IMOEX               False
LOW_IMOEX                False
VALUE_IMOEX              False
CAPITALIZATION_IMOEX     False
CLOSE_MOEXMM             False
OPEN_MOEXMM              False
HIGH_MOEXMM              False
LOW_MOEXMM               False
VALUE_MOEXMM             False
CAPITALIZATION_MOEXMM    False
Key_rate                 False
Inflation                False
USD_RUB                  False
Price_GOLD               False
Day_type                 False
month                    False
dtype: bool

In [35]:
IMOEX_df.to_csv('IMOEX_df', index=False)
MOEXMM_df.to_csv('MOEXMM_df', index=False)
GOLD_df.to_csv('GOLD_df', index=False)
USDRUB_df.to_csv('USDRUB_df', index=False)
INFLATION_df.to_csv('INFLATION_df', index=False)
RESULT_df.to_csv('RESULT_df', index=False)